In [1]:
# THis notebook creates a subset of 'Singles' of the Taiwan Credit Default dataset. 
# Import Data handling/display libraries
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler, RobustScaler 
from typing import List, Union, Dict
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the Taiwan Credit Card Default training dataset
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Default.csv')

# Drop the first row because it is a redundant heading row.
Taiwan_df.to_csv('./input/Taiwan-Credit-Card-New.csv', header=False, index=False)

# Re-read the dataset with its proper column headers
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-New.csv')
# Rename a couple of columns for aesthetics
Taiwan_df.rename(columns={'SEX':'GENDER', 'PAY_0': 'PAY_1',
                          'default payment next month':'DEFAULT'}, 
                          inplace=True)
# Drop ID as it has no impact on this study.
Taiwan_df.drop(["ID"], axis=1, inplace=True)

In [3]:
# Make males 0, Females 1
gender = {1: 0,2: 1} 
Taiwan_df.GENDER = [gender[item] for item in Taiwan_df.GENDER]
Taiwan_df.GENDER.value_counts()

1    18112
0    11888
Name: GENDER, dtype: int64

In [4]:
# Create a subset of all MARRIAGE== 2 (singles) of the dataset
Taiwan_df = Taiwan_df.loc[Taiwan_df['MARRIAGE'] ==2]
Taiwan_df.head(5)

,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
1,120000,1,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,1,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
5,50000,0,1,2,37,0,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
6,500000,0,1,2,29,0,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
7,100000,1,2,2,23,0,-1,-1,0,0,...,221,-159,567,380,601,0,581,1687,1542,0


In [5]:
Taiwan_df.GENDER.value_counts()

1    9411
0    6553
Name: GENDER, dtype: int64

In [6]:
Taiwan_df.shape

(15964, 24)

In [7]:
# We create a new column AMT_PAY_RATIO to hold the ratio of:
# Total sum owed / Total sum paid over 6 months
Taiwan_df['AMT_PAY_RATIO'] = (Taiwan_df['BILL_AMT1']+Taiwan_df['BILL_AMT2']+
                              Taiwan_df['BILL_AMT3']+Taiwan_df['BILL_AMT4']+
                              Taiwan_df['BILL_AMT5']+
                              Taiwan_df['BILL_AMT6']) / (Taiwan_df['PAY_AMT1']+
                                                         Taiwan_df['PAY_AMT2']+Taiwan_df['PAY_AMT3']+
                                                         Taiwan_df['PAY_AMT4']+Taiwan_df['PAY_AMT5']+
                                                         Taiwan_df['PAY_AMT6'])  

In [8]:
# Here we create a 6 new columns called AMT-PAY-RATIO. This will contain 
# the amount/payment ratio for the 6 BILL_AMTx/PAY_AMTx. 
Taiwan_df['AMT_PAY_RATIO1'] = Taiwan_df['BILL_AMT1']/Taiwan_df['PAY_AMT1']
Taiwan_df['AMT_PAY_RATIO2'] = Taiwan_df['BILL_AMT2']/Taiwan_df['PAY_AMT2']
Taiwan_df['AMT_PAY_RATIO3'] = Taiwan_df['BILL_AMT3']/Taiwan_df['PAY_AMT3']
Taiwan_df['AMT_PAY_RATIO4'] = Taiwan_df['BILL_AMT4']/Taiwan_df['PAY_AMT4']
Taiwan_df['AMT_PAY_RATIO5'] = Taiwan_df['BILL_AMT5']/Taiwan_df['PAY_AMT5']
Taiwan_df['AMT_PAY_RATIO6'] = Taiwan_df['BILL_AMT6']/Taiwan_df['PAY_AMT6']

# We now replace the inf values (from division by zero) with NaN
Taiwan_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# We then drop the all the original BILL_AMTx aand PAY_AMTx columns.
Taiwan_df.drop(["BILL_AMT1","BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6"], axis=1, inplace=True)
Taiwan_df.drop(["PAY_AMT1", "PAY_AMT2" , "PAY_AMT3" , "PAY_AMT4" , "PAY_AMT5" , "PAY_AMT6" ], axis=1, inplace=True)
#Taiwan_df.head(10)

In [9]:
# Now we do some data cleansing:
# As the Education categories 0, 5, and 6 have no definition, 
# we shall make them 'Education Other', eg '4'
fil = (Taiwan_df.EDUCATION == 5) | (Taiwan_df.EDUCATION == 6) | (Taiwan_df.EDUCATION == 0)
Taiwan_df.loc[fil, 'EDUCATION'] = 4
# Same with MARRIAGE category 0 - it has no definition, 
# so we make them 'Marriage Other', eg '3'
Taiwan_df.loc[Taiwan_df.MARRIAGE == 0, 'MARRIAGE'] = 3
# Taiwan_df.MARRIAGE.value_counts()
#Taiwan_df.head(10)

In [10]:
# Create a function to return columns from the dataframe.
class SelectCols(TransformerMixin):
    """Select columns from a DataFrame."""
    def __init__(self, cols: List[str]) -> None:
        self.cols = cols
    def fit(self, x: None) -> "SelectCols":
        """Nothing to do."""
        return self
    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        """Return just selected columns."""
        return x[self.cols]

In [11]:
class NumericEncoder(TransformerMixin):
    """Remove invalid values from numerical columns, replace with median."""
    def fit(self, x: pd.DataFrame) -> "NumericEncoder":
        # Find median for all columns. Examine all columns for non-NaNs, invalid values and NaNs
        # If a column is all NaNs (after coercion), the median value will be a NaN.
        self.encoders_ = {
            c: pd.to_numeric(x[c], errors='coerce').median(skipna=True) for c in x}
        return self

    def transform(self, x: pd.DataFrame) -> pd.DataFrame:
        #For each column in x, encode any NaN values with the learned medium 
        # Create a list of new DataFrames, each with 1 cleaned columns
        output_dfs = []
        for c in x:
            new_cols = pd.DataFrame()
            # Find invalid values that aren't nans (-inf, inf, string)
            invalid_idx = pd.to_numeric(x[c].replace([-np.inf, np.inf],np.nan),
                                        errors='coerce').isnull()
            # Copy to new df for this column
            new_cols.loc[:, c] = x[c].copy()
            # Replace the invalid values with learned median
            new_cols.loc[invalid_idx, c] = self.encoders_[c]
            output_dfs.append(new_cols)

        # Concat list of output_dfs to single df
        df = pd.concat(output_dfs, axis=1)
        # Return with an remaining NaNs removed. These might exist if the median
        # is a NaN because there was no numeric data in the column at all.
        return df.fillna(0)

In [12]:
# Construct the data engineering pipeline
# NumericEncoding fork: Select numeric columns -> numeric encode
pp_numeric_cols = Pipeline([('select', SelectCols(cols=['LIMIT_BAL','GENDER','EDUCATION','MARRIAGE','AGE',
                                                       'PAY_1','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6',
                                                       'AMT_PAY_RATIO1', 'AMT_PAY_RATIO2', 'AMT_PAY_RATIO3', 
                                                       'AMT_PAY_RATIO4', 'AMT_PAY_RATIO5', 'AMT_PAY_RATIO6',
                                                       'AMT_PAY_RATIO', 'DEFAULT'])),
                            ('process', NumericEncoder())])

In [13]:
# .fit_transform on the Taiwan dataset
Taiwan_pp = pp_numeric_cols.fit_transform(Taiwan_df)
# Test step - dropping columns to see whether we get meaningful results in de-biasing.
# Taiwan_pp.drop(["PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"], axis=1, inplace=True)
# Taiwan_pp.drop(["AMT_PAY_RATIO1","AMT_PAY_RATIO2", "AMT_PAY_RATIO3", "AMT_PAY_RATIO4", "AMT_PAY_RATIO5", "AMT_PAY_RATIO6"], axis=1, inplace=True)
# Taiwan_pp.drop(["LIMIT_BAL", "AMT_PAY_RATIO"], axis=1, inplace=True)
# Taiwan_pp.drop(["PAY_1", "PAY_2" , "PAY_3" , "PAY_4" , "PAY_5" , "PAY_6" ], axis=1, inplace=True)
# Taiwan_pp.drop(["PAY_1", "PAY_2" , "PAY_3" , "PAY_4" , "PAY_5" , "PAY_6" ], axis=1, inplace=True)
#Taiwan_pp.head(10)

In [14]:
# Finally write the cleaned dataset out to a csv
# Taiwan_scaled_df.to_csv('./input/Taiwan-Credit-Card-Cleaned.csv',index=False)
Taiwan_pp.to_csv('./input/Taiwan-Credit-Card-Cleaned-Subset.csv',index=False)
# Taiwan_df.to_csv('./input/Taiwan-Credit-Card-Cleaned.csv',index=False)